## Google Calendar API Testing

In [84]:
from gcsa.google_calendar import GoogleCalendar
from gcsa.calendar import Calendar
import time
from datetime import datetime, timedelta

Using the Google Calendar API, I need to figure out some way of: 

* Based on the user-selected orgs, obtain the Google Calendars of those orgs
  * The links to each org's calendar should be stored in a database / CSV of some sort, but it also needs to be updateable on the org's side in case they make a new calendar.
* Consolidate the selected calendars into a SINGLE calendar where:
  * Each different club's events has a different colors
    * But if colors don't work out, definitely label each event w the club name
  * The events are SYNCED over time - if a club adds an event to their individual calendar, this should be reflected on the consolidated calendar. I'm actually a bit concerned as to whether this will work out.
* Once the calendars have been consolidated, generate a link that the user can add to their Google Calendar (i.e. subscribe to the consolidated calendar) or generate a .ics file - not sure what the difference is tbh. 

In [85]:
# using the credentials from a past gcal project
gc = GoogleCalendar(credentials_path="/home/jasmine/PROJECTS/hornsync/credentials.json")
gc

I currently have the WICS gcal and the Austin Bodybuilding gcal. I'll use these as examples.

In [86]:
wics_id = "k06u9ic0ke0bgoq3n7im2sibsc@group.calendar.google.com"
abb_id = "225cdda94148e6ef2aa3a095ca760d12cef989b31795c2044b9828c5c26f8bc7@group.calendar.google.com"

In [87]:
for event in gc.get_events(calendar_id=wics_id): 
    print(event)

2025-02-09 08:00:00-06:00 - WiCS Hack 2025 Day 2
2025-02-22 10:00:00-06:00 - UT STEM Girl Day Volunteering
2025-02-08 08:00:00-06:00 - WiCS Hacks 2025 Day 1


In [88]:
for event in gc.get_events(calendar_id=abb_id): 
    print(event)

2024-07-19 - Aidan’s bday
2025-01-30 19:00:00-06:00 - General Meeting
2025-02-13 19:00:00-06:00 - General Meeting 2
2025-02-27 19:00:00-06:00 - General Meeting 3
2025-03-13 19:00:00-05:00 - General Meeting 4
2025-03-27 19:00:00-05:00 - General Meeting 5
2025-04-10 19:00:00-05:00 - General Meeting 6
2025-04-24 19:00:00-05:00 - General Meeting 7
2025-04-12 21:00:00-05:00 - ABB Formal
2025-05-03 11:30:00-05:00 - ABB Barge
2025-03-08 22:00:00-06:00 - Mardi Gras Party
2025-02-06 - FAMILY CHALLENGE #1
2025-02-20 - FAMILY CHALLENGE #1
2025-03-06 - FAMILY CHALLENGE #2
2025-03-20 - FAMILY CHALLENGE #3
2025-04-03 - FAMILY CHALLENGE #4
2025-04-17 - FAMILY CHALLENGE #5
2025-05-01 - FAMILY CHALLENGE #6
2025-03-22 11:00:00-05:00 - Olympics / Field Day w Longhorn Powerlifting DATE TBD, PLACEHOLDER
2025-02-15 22:00:00-06:00 - Flex & Flirt Party (Valentine’s)
2025-02-12 - Fees Due $45
2025-02-13 17:00:00-06:00 - Cabo Bob's Profit Share


The above seems to fetch all future events? Might be nice if on the user end the user can specify a date range to fetch events from, where the default is for future events but there is the option if you want past events as well.

In [89]:
TODAY = datetime.today().date()
TODAY

datetime.date(2025, 2, 8)

In [90]:
for event in gc.get_events(calendar_id=abb_id, 
                           time_min=TODAY): 
    print(event)

2024-07-19 - Aidan’s bday
2025-01-30 19:00:00-06:00 - General Meeting
2025-02-13 19:00:00-06:00 - General Meeting 2
2025-02-27 19:00:00-06:00 - General Meeting 3
2025-03-13 19:00:00-05:00 - General Meeting 4
2025-03-27 19:00:00-05:00 - General Meeting 5
2025-04-10 19:00:00-05:00 - General Meeting 6
2025-04-24 19:00:00-05:00 - General Meeting 7
2025-04-12 21:00:00-05:00 - ABB Formal
2025-05-03 11:30:00-05:00 - ABB Barge
2025-03-08 22:00:00-06:00 - Mardi Gras Party
2025-02-06 - FAMILY CHALLENGE #1
2025-02-20 - FAMILY CHALLENGE #1
2025-03-06 - FAMILY CHALLENGE #2
2025-03-20 - FAMILY CHALLENGE #3
2025-04-03 - FAMILY CHALLENGE #4
2025-04-17 - FAMILY CHALLENGE #5
2025-05-01 - FAMILY CHALLENGE #6
2025-03-22 11:00:00-05:00 - Olympics / Field Day w Longhorn Powerlifting DATE TBD, PLACEHOLDER
2025-02-15 22:00:00-06:00 - Flex & Flirt Party (Valentine’s)
2025-02-12 - Fees Due $45
2025-02-13 17:00:00-06:00 - Cabo Bob's Profit Share


In [100]:
# simulating a user who has selected WICS and ABB as the clubs they are interested in
ORG_DICT = {
    "WICS": wics_id, 
    "ABB": abb_id
}

# the clubs the user has selected
user_clubs = ["WICS", "ABB"]

# list to store ALL events across the user's selected clubs
EVENTS = []

for i, selected_club in enumerate(user_clubs): 
    for event in gc.get_events(calendar_id=ORG_DICT[selected_club]): 
        event.color_id = i
        event.summary = f"[{selected_club}] - {event.summary}"
        EVENTS.append(event)

EVENTS

[<Event 2025-02-09 08:00:00-06:00 - [WICS] - WiCS Hack 2025 Day 2>,
 <Event 2025-02-22 10:00:00-06:00 - [WICS] - UT STEM Girl Day Volunteering>,
 <Event 2025-02-08 08:00:00-06:00 - [WICS] - WiCS Hacks 2025 Day 1>,
 <Event 2024-07-19 - [ABB] - Aidan’s bday>,
 <Event 2025-01-30 19:00:00-06:00 - [ABB] - General Meeting>,
 <Event 2025-02-13 19:00:00-06:00 - [ABB] - General Meeting 2>,
 <Event 2025-02-27 19:00:00-06:00 - [ABB] - General Meeting 3>,
 <Event 2025-03-13 19:00:00-05:00 - [ABB] - General Meeting 4>,
 <Event 2025-03-27 19:00:00-05:00 - [ABB] - General Meeting 5>,
 <Event 2025-04-10 19:00:00-05:00 - [ABB] - General Meeting 6>,
 <Event 2025-04-24 19:00:00-05:00 - [ABB] - General Meeting 7>,
 <Event 2025-04-12 21:00:00-05:00 - [ABB] - ABB Formal>,
 <Event 2025-05-03 11:30:00-05:00 - [ABB] - ABB Barge>,
 <Event 2025-03-08 22:00:00-06:00 - [ABB] - Mardi Gras Party>,
 <Event 2025-02-06 - [ABB] - FAMILY CHALLENGE #1>,
 <Event 2025-02-20 - [ABB] - FAMILY CHALLENGE #1>,
 <Event 2025-03-06

In [101]:
for e in EVENTS: 
    print(e.color_id)

0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [102]:
gc.list_event_colors()

{'1': {'background': '#a4bdfc', 'foreground': '#1d1d1d'},
 '2': {'background': '#7ae7bf', 'foreground': '#1d1d1d'},
 '3': {'background': '#dbadff', 'foreground': '#1d1d1d'},
 '4': {'background': '#ff887c', 'foreground': '#1d1d1d'},
 '5': {'background': '#fbd75b', 'foreground': '#1d1d1d'},
 '6': {'background': '#ffb878', 'foreground': '#1d1d1d'},
 '7': {'background': '#46d6db', 'foreground': '#1d1d1d'},
 '8': {'background': '#e1e1e1', 'foreground': '#1d1d1d'},
 '9': {'background': '#5484ed', 'foreground': '#1d1d1d'},
 '10': {'background': '#51b749', 'foreground': '#1d1d1d'},
 '11': {'background': '#dc2127', 'foreground': '#1d1d1d'}}

In [103]:
# create consolidated calendar 
consolidated_cal = Calendar("Your Hornsync Clubs", 
                            description="Events for your selected clubs via Hornsync")

consolidated_cal

<Calendar Your Hornsync Clubs - Events for your selected clubs via Hornsync>

In [104]:
consolidated_cal = gc.add_calendar(consolidated_cal)
consolidated_cal

<Calendar Your Hornsync Clubs - Events for your selected clubs via Hornsync>

In [105]:
for c in gc.get_calendar_list():
    print(c)

None - (Holidays in China)
iCloud - (jasminexu7003@icloud.com)
Celtics - (Boston Celtics 2023-24 Schedule)
Texas MBB - (Men's Basketball - texassports.com)
Texas Volleyball - (Volleyball - texassports.com)
Texas Football - (Football - texassports.com)
None - (Footy/Games)
None - (Premier League)
None - (Austin Bodybuilding)
None - (WiCS Events)
None - (Spurs games)
None - (School)
None - (Classes)
None - (Blocks)
None - (Birthdays)
None - (jasminexu@utexas.edu)
None - (Holidays in United States)
None - (Your Hornsync Clubs)


In [106]:
for e in EVENTS: 
    gc.import_event(e, calendar_id=c.id)
    # time.sleep(1)

In [71]:
gc.get_calendar().id

'jasminexu@utexas.edu'

In [72]:
c.id

'c_cfa9c1e996673fb09fb5159a9edfd261af275123343126a877c488e849ccedcb@group.calendar.google.com'

In [98]:
for event in gc.get_events(calendar_id=c.id): 
    print(event)

2025-02-13 19:00:00-06:00 - General Meeting 2
2025-02-27 19:00:00-06:00 - General Meeting 3
2025-03-13 19:00:00-05:00 - General Meeting 4
2025-03-27 19:00:00-05:00 - General Meeting 5
2025-04-10 19:00:00-05:00 - General Meeting 6
2025-04-24 19:00:00-05:00 - General Meeting 7
2025-04-12 21:00:00-05:00 - ABB Formal
2025-05-03 11:30:00-05:00 - ABB Barge
2025-03-08 22:00:00-06:00 - Mardi Gras Party
2025-02-06 - FAMILY CHALLENGE #1
2025-02-20 - FAMILY CHALLENGE #1
2025-03-06 - FAMILY CHALLENGE #2
2025-03-20 - FAMILY CHALLENGE #3
2025-04-03 - FAMILY CHALLENGE #4
2025-04-17 - FAMILY CHALLENGE #5
2025-05-01 - FAMILY CHALLENGE #6
2025-03-22 11:00:00-05:00 - Olympics / Field Day w Longhorn Powerlifting DATE TBD, PLACEHOLDER
2025-02-15 22:00:00-06:00 - Flex & Flirt Party (Valentine’s)
2025-02-12 - Fees Due $45
2025-02-13 17:00:00-06:00 - Cabo Bob's Profit Share


In [78]:
gc.creds

AttributeError: 'GoogleCalendar' object has no attribute 'creds'

In [79]:
from gcsa.google_calendar import GoogleCalendar
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

# Path to your OAuth 2.0 credentials JSON file
credentials_path = "/home/jasmine/PROJECTS/hornsync/credentials.json"

# Set up GoogleCalendar client with credentials
gc = GoogleCalendar(credentials_path=credentials_path)

# If the credentials have expired, refresh them
if gc.creds and gc.creds.expired and gc.creds.refresh_token:
    gc.creds.refresh(Request())

AttributeError: 'GoogleCalendar' object has no attribute 'creds'